# APIM ❤️ OpenAI

## OpenAI Mock Server(s)

This notebook provides guidance on the use of mock servers locally and in Azure. The AI-Gateway Mock servers are designed to mimic the behavior and responses of the OpenAI API, thereby creating an efficient simulation environment suitable for testing and development purposes on the integration with APIM and other use cases.
The ```app.py``` file can be customized to tailor the Mock server to your unique requirements.

### Run locally
Open a terminal and type:
```
pip install -r requirements.txt
flask --app app.py --debug run
```
### Deploy to Azure Web Apps
Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

Execute the following steps to deploy to Azure.
The ```mock_webapps``` is a list of names that will be used to create the Azure Web App(s). All the Web Apps will reuse the same App Service Plan.

### 0️⃣ Initialize notebook variables

In [17]:
mock_resource_group = "lab-ai-gateway-mock"
mock_location = "swedencentral" # the location that will be used for the resource group, app service plan and web apps
mock_webapps = [ {"name": "openaimock1"}, {"name": "openaimock2"} ] # ensure that the names are not being used within Azure
mock_webapp_runtime = "PYTHON:3.12"
mock_app_service_plan = "openaimock-plan"
mock_app_service_plan_sku = "B1"


### 1️⃣ Deploy or update Mock Server(s)
The following command is an [all-in-one command](https://learn.microsoft.com/en-us/cli/azure/webapp?view=azure-cli-latest#az-webapp-up) that creates all the Azure resources and deploys the source code. When finished it should open a browser window for each Mock server created.
You can repeat this step if you modify the ```app.py``` file to incorporate your specific mocking behavior. This will allow you to redeploy using the same resources that were previously created. 

In [18]:
for mock_webapp in mock_webapps:
    mock_webapp_name = mock_webapp.get("name")
    ! az webapp up --resource-group {mock_resource_group} --name {mock_webapp_name} --location {mock_location} \
        --plan {mock_app_service_plan} --runtime {mock_webapp_runtime} --sku {mock_app_service_plan_sku} --track-status true --launch-browser 

{
  "appserviceplan": "openaimock-plan",
  "location": "swedencentral",
  "name": "openaimock1",
  "os": "Linux",
  "resourcegroup": "lab-ai-gateway-mock",
  "runtime_version": "PYTHON|3.12",
  "runtime_version_detected": "-",
  "sku": "BASIC",
  "src_path": "c:\\Projects\\AI-Gateway\\mock-server"
}


D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\OpenSSL/_util.py:6: UserWarning: You are using cryptography on a 32-bit Python on a 64-bit Windows Operating System. Cryptography will be significantly faster if you switch to using a 64-bit Python.


{
  "appserviceplan": "openaimock-plan",
  "location": "swedencentral",
  "name": "openaimock2",
  "os": "Linux",
  "resourcegroup": "lab-ai-gateway-mock",
  "runtime_version": "PYTHON|3.12",
  "runtime_version_detected": "-",
  "sku": "BASIC",
  "src_path": "c:\\Projects\\AI-Gateway\\mock-server"
}


D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\OpenSSL/_util.py:6: UserWarning: You are using cryptography on a 32-bit Python on a 64-bit Windows Operating System. Cryptography will be significantly faster if you switch to using a 64-bit Python.


### 2️⃣ Test the Mock server(s)
The following code uses the Python HTTP client library to simulate OpenAI requests to the Mock server(s). To guide the mock server towards simulating behaviors such as returning a specific status code or incorporating a delay to influence the response time, you may modify the content structure as illustrated below.

In [25]:
import json
import requests

for mock_webapp in mock_webapps:
    mock_webapp_name = mock_webapp.get("name")
    url = "https://" + mock_webapp_name + ".azurewebsites.net/openai/deployments/gpt5/chat/completions?api-version=2024-02-01"
    messages={
        "messages": [
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0}
                    }
                }
            }
        ]
    }
    response = requests.post(url, json = messages)
    print("status code: ", response.status_code)
    print("headers: ", response.headers)
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)

status code:  429
headers:  {'Content-Length': '85', 'Content-Type': 'application/json', 'Date': 'Tue, 09 Apr 2024 12:53:06 GMT', 'Server': 'gunicorn', 'Set-Cookie': 'ARRAffinity=66379d7aad515b9bbf7999f46645cae8f2e2a47fd276f9546c5765e57c056668;Path=/;HttpOnly;Secure;Domain=openaimock1.azurewebsites.net, ARRAffinitySameSite=66379d7aad515b9bbf7999f46645cae8f2e2a47fd276f9546c5765e57c056668;Path=/;HttpOnly;SameSite=None;Secure;Domain=openaimock1.azurewebsites.net', 'x-ms-region': 'openaimock1.azurewebsites.net'}
{"error":{"code":"429","message":"Rate limit is exceeded. Try again in 5 seconds."}}

status code:  200
headers:  {'Content-Length': '308', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Tue, 09 Apr 2024 12:53:06 GMT', 'Server': 'gunicorn', 'Set-Cookie': 'ARRAffinity=66379d7aad515b9bbf7999f46645cae8f2e2a47fd276f9546c5765e57c056668;Path=/;HttpOnly;Secure;Domain=openaimock2.azurewebsites.net, ARRAffinitySameSite=66379d7aad515b9bbf7999f46645cae8f2e2a47fd276f9546c5765e57c056668;P

### (optional) Stream logs

Execute the result of the following script in a terminal to tail the streamed logs

In [20]:
for mock_webapp in mock_webapps:
    mock_webapp_name = mock_webapp.get("name")
    ! echo az webapp log tail --name {mock_webapp_name} --resource-group {mock_resource_group}

az webapp log tail --name openaimock1 --resource-group lab-ai-gateway-mock


az webapp log tail --name openaimock2 --resource-group lab-ai-gateway-mock


### 🗑️ Clean up resources

When you're finished with the APIM ❤️ OpenAI Mock server(s), you can remove all your deployed resources from Azure. To avoid extra charges and keep your Azure subscription uncluttered. Removing the resource group is the fastest way to remove all Azure resources that you have created.

In [21]:
run_cell = False
if run_cell:
    ! az group delete --name {mock_resource_group} -y